In [ ]:
%%capture
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "langchain==0.2.11" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-core==0.2.43" --user

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_ibm import WatsonxLLM

granite_llm = WatsonxLLM(
    model_id="ibm/granite-3-2-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params={
        "max_new_tokens": 256,
        "temperature": 0.5,
        "top_p": 0.2
    }
)

In [ ]:
def llm_model(prompt_txt, params=None):
    model_id = "ibm/granite-3-2-8b-instruct"
    default_params = {
        "max_new_tokens": 256,
        "temperature": 0.5,
        "top_p": 0.2
    }
    if params:
        default_params.update(params)

    granite_llm = WatsonxLLM(
        model_id=model_id,
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params=default_params
    )

    response = granite_llm.invoke(prompt_txt)
    return response

In [ ]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

GenParams().get_example_values()

parameters = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.5,
}

In [ ]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 10,
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1
}
prompt = "The wind is"

response = llm_model(prompt, params)
print(prompt)
print(response)

In [ ]:
prompt = """Classify the following statement as true or false:
'The Eiffel Tower is located in Berlin.'
Answer:
"""

response = llm_model(prompt, params)
print(response)

In [ ]:
params = {
    "max_new_tokens": 20,
    "temperature": 0.1,
}
prompt = """Here is an example of translating a sentence from English to French:

English: "How is the weather today?"
French: "Comment est le temps aujourd'hui?"

Now translate:
English: "Where is the nearest supermarket?"
"""

response = llm_model(prompt, params)
print(response)

In [ ]:
params = {"max_new_tokens": 10}
prompt = """Here are few examples of classifying emotions:

Statement: I just won my first marathon!
Emotion: Joy

Statement: I can’t believe I lost my keys again.
Emotion: Frustration

Statement: My best friend is moving.
Emotion: Sadness

Now classify:
Statement: That movie was so scary I had to cover my eyes.
"""

print(llm_model(prompt, params))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

template = """Tell me a {adjective} joke about {content}."""
prompt = PromptTemplate.from_template(template)

def format_prompt(vars):
    return prompt.format(**vars)

chain = (
    RunnableLambda(format_prompt)
    | granite_llm
    | StrOutputParser()
)

print(chain.invoke({"adjective":"funny","content":"chickens"}))